In [1]:
import pandas as pd

retriever_sample = pd.read_csv(
    "data/retriever_sample.csv", low_memory=False, keep_default_na=False
)
retriever_sample["setup"].value_counts()

 1    301
-1     46
 0     26
Name: setup, dtype: int64

In [2]:
retriever_sample["rank"].value_counts()

 1      315
 2       34
-1        8
 4        7
 3        3
 7        2
 13       1
 16       1
 5        1
 290      1
Name: rank, dtype: int64

In [3]:
retriever_sample["first"].value_counts()

 1    360
 0      7
-1      6
Name: first, dtype: int64

In [4]:
import json
import pandas as pd

retriever_sample = pd.read_csv(
    "data/retriever_dataset.csv", low_memory=False, keep_default_na=False
)
final = json.load(open("data/retriever_sample_final.json"))
retriever_sample["final"] = retriever_sample["name"].map(final).fillna("")
len(final), len(retriever_sample)

(12367, 12375)

In [6]:
from Levenshtein import ratio
import re

sub_pattern = re.compile(r"[^a-zA-Z0-9]")


def cal_ratio(x):
    name = x["name"]
    final = x["final"]
    if final == "":
        return -1
    user, repo = final.split("/")[-2:]
    ratio1 = ratio(sub_pattern.sub("", name), sub_pattern.sub("", repo))
    ratio2 = ratio(sub_pattern.sub("", name), sub_pattern.sub("", user + repo))
    return max(ratio1, ratio2)


retriever_sample["ratio"] = retriever_sample.apply(cal_ratio, axis=1)

In [7]:
for i in range(11):
    ratio = i / 10
    retrieved = retriever_sample[(retriever_sample["ratio"] >= ratio)]
    retrieved_correct = retriever_sample[
        (retriever_sample["ratio"] >= ratio)
        & (retriever_sample["repo_url"] == retriever_sample["final"])
    ]
    print(
        ratio,
        len(retrieved),
        len(retrieved) / len(retriever_sample),
        len(retrieved_correct),
        len(retrieved_correct) / len(retrieved),
    )

0.0 12202 0.986020202020202 11353 0.9304212424192755
0.1 12185 0.9846464646464647 11351 0.9315551908083709
0.2 12074 0.9756767676767677 11334 0.9387112804373033
0.3 11808 0.9541818181818181 11261 0.9536754742547425
0.4 11476 0.9273535353535354 11078 0.9653189264552109
0.5 11165 0.9022222222222223 10834 0.9703537841468876
0.6 10845 0.8763636363636363 10576 0.9751959428307976
0.7 10429 0.8427474747474748 10205 0.9785214306261386
0.8 9935 0.8028282828282828 9760 0.9823855057876195
0.9 9505 0.768080808080808 9365 0.9852709100473435
1.0 9370 0.7571717171717172 9243 0.9864461045891142


In [9]:
import pandas as pd

df = pd.read_csv(
    "data/retriever_dataset_remaining.csv", low_memory=False, keep_default_na=False
)
print(len(df))
df.head()

81751


,name,version,metadata_retriever,final
0,able,0.3.3,,
1,tapdance,0.9.9.dev265,,https://github.com/dataops-tk/tapdance
2,dash-mp-components,0.3.42,,https://github.com/materialsproject/dash-mp-co...
3,dbt-cratedb,0.20.0a65,,https://github.com/usnistgov/dft-crossfilter
4,metaphor-models,0.5.4,,


In [10]:
from Levenshtein import ratio
import re

sub_pattern = re.compile(r"[^a-zA-Z0-9]")


def cal_ratio(x):
    name = x["name"]
    final = x["final"]
    if final == "":
        return -1
    user, repo = final.split("/")[-2:]
    ratio1 = ratio(sub_pattern.sub("", name), sub_pattern.sub("", repo))
    ratio2 = ratio(sub_pattern.sub("", name), sub_pattern.sub("", user + repo))
    return max(ratio1, ratio2)


df["ratio"] = df.apply(cal_ratio, axis=1)

In [11]:
for i in range(11):
    ratio = i / 10
    print(ratio, len(df[df["ratio"] >= ratio]))

0.0 52359
0.1 51957
0.2 49266
0.3 41727
0.4 34744
0.5 32139
0.6 29966
0.7 27755
0.8 25364
0.9 23134
1.0 22190


In [13]:
df[df["ratio"] >= 0.5][["name", "version", "final"]].sample(
    100, random_state=42
).to_csv("data/retriever_evaluation.csv", index=False)

In [16]:
evaluation_df = pd.read_csv("data/retriever_evaluation.csv", keep_default_na=False)
evaluation_df["label"].value_counts()

Correct      90
Incorrect    10
Name: label, dtype: int64